This notebook is only for testing. Maybe we can build a working model from this, and then we can start putting code into the "model" python module.

In [1]:
import tensorflow as tf
import numpy as np
import time
import importlib
import os
import re

In [2]:
import sys
sys.path.append('../py')
import cavitylearn.data
import cavitylearn.catalonet0

## Loading Data

In [3]:
dataset_basedir = '/var/cavitylearn/dataset-test/'

In [5]:
dataconfig = cavitylearn.data.read_dataconfig(os.path.join(dataset_basedir, "datainfo.ini"))
boxfiles = [e.path for e in os.scandir(os.path.join(dataset_basedir, "boxes")) if e.is_file()]
with open(os.path.join(dataset_basedir, "labels.txt"), 'rt') as labelfile:
    trainset = cavitylearn.data.DataSet(labelfile, boxfiles, dataconfig)

In [6]:
print('Examples available in dataset: ', trainset.N)

Examples available in dataset:  21687


## Building the model

In [7]:
boxes_placeholder = tf.placeholder(tf.float32, shape=[None, dataconfig.boxshape[0], dataconfig.boxshape[1], dataconfig.boxshape[2], dataconfig.num_props])
labels_placholder = tf.placeholder(tf.float32, shape=[None, dataconfig.num_classes])

In [8]:
logits = cavitylearn.catalonet0.inference(boxes_placeholder, dataconfig)

In [9]:
loss = cavitylearn.catalonet0.loss(logits, labels_placholder)

In [10]:
global_step = tf.Variable(0, name='global_step', trainable=False)

In [11]:
train_op = cavitylearn.catalonet0.train(loss, 1e-4, global_step)

In [12]:
correct_prediction = tf.equal(tf.argmax(logits,1),  tf.argmax(labels_placholder,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name="accuracy")
tf.scalar_summary("accuracy", accuracy)

<tf.Tensor 'ScalarSummary_1:0' shape=() dtype=string>

## Training the Model

In [13]:
def purge(directory, pattern):
    for f in os.listdir(directory):
        if re.search(pattern, f):
            os.remove(os.path.join(directory, f))

In [14]:
saver = tf.train.Saver()

In [15]:
summary_op = tf.merge_all_summaries()

In [19]:
CONTINUE = True
BATCHSIZE = 50
NUM_BATCHES = 10
run_name = "run0"
CHECKPOINT_FREQUENCY = 20

In [22]:
checkpoint_dir = os.path.join(dataset_basedir, "checkpoints", run_name)
run_dir = dataset_basedir
trainset.rewind_batches()
with tf.Session() as sess:
    if not os.path.isdir(os.path.join(run_dir, "logs", run_name)):
        os.makedirs(os.path.join(run_dir, "logs", run_name))
    
    # purge log directory for run, before running it again
    purge(os.path.join(dataset_basedir, "logs", run_name), r'^events\.out\.tfevents\.\d+')
    
    # Create summary writer
    summary_writer = tf.train.SummaryWriter(os.path.join(dataset_basedir, "logs", run_name), sess.graph)
    
    
    sess.run(tf.initialize_all_variables())
    
    start_time = time.time()
    for i in range(NUM_BATCHES):
        # get training data
        labels, boxes = trainset.next_batch(BATCHSIZE)
        
        # abort training if we didn't get any more data
        if len(labels) == 0:
            break
        
        feed_dict = {
            boxes_placeholder: boxes,
            labels_placholder: labels
        }
        
        # Do it!
        sess.run(train_op, feed_dict=feed_dict)
        print('.', end="")
        
        
        summary_str, step = sess.run([summary_op, global_step], feed_dict=feed_dict)
        summary_writer.add_summary(summary_str, step)
        summary_writer.flush()
        
        if i % CHECKPOINT_FREQUENCY == 0:
            saver.save(sess, checkpoint_dir, global_step=global_step)
            
        
    saver.save(sess, checkpoint_dir, global_step=global_step)
    end_time = time.time()

..........

In [23]:
print("Total time: ", str(end_time - start_time))
print("Time per batch: ", str((end_time - start_time)/NUM_BATCHES))

Total time:  10.981575727462769
Time per batch:  1.098157572746277


importlib.reload(cavitylearn.catalonet0)
tf.reset_default_graph()